# DataLoader Setup

In [1]:
import os
import torch
import torchvision
from pytorch_lightning import seed_everything
from pl_bolts.datamodules import CIFAR10DataModule, TinyCIFAR10DataModule
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization

PATH_DATASETS = os.environ.get('PATH_DATASETS', '../datasets')
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 512 if AVAIL_GPUS else 64
NUM_WORKERS = int(os.cpu_count()/2)
TEST_ONLY = False

test_transforms = [
        torchvision.transforms.ToTensor(),
        cifar10_normalization(),        
        torchvision.transforms.Lambda(lambda x: x.clamp(min = -1, max = 1)),
        torchvision.transforms.Lambda(lambda x: x * 127),   
        torchvision.transforms.Lambda(lambda x: x.floor())  
]

train_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.RandomCrop(32, padding=4),
        torchvision.transforms.RandomHorizontalFlip(),
    ] 
    + test_transforms
)

test_transforms = torchvision.transforms.Compose(test_transforms)

cifar10_dm = CIFAR10DataModule(
    data_dir = PATH_DATASETS,
    batch_size = BATCH_SIZE,
    num_workers = NUM_WORKERS,
    train_transforms = train_transforms,
    test_transforms = test_transforms,
    val_transforms = test_transforms,
)

seed_everything(seed=1234, workers=True)

/home/neungyun/anaconda3/envs/torch/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:73: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
/home/neungyun/anaconda3/envs/torch/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:77: LightningDeprecationWarning: DataModule property `val_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
/home/neungyun/anaconda3/envs/torch/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:81: LightningDeprecationWarning: DataModule property `test_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
Global seed set to 1234


1234

# model 

In [2]:
from model import ReactModel
from model import TeacherModel

base_model = [{'in_channels':3, 'out_channels':32, 'stride':1 }, # block : react or baseline
            {'in_channels':32, 'out_channels':64, 'block':'baseline'}, # 16
            {'in_channels':64, 'out_channels':128, 'block':'baseline'}, # 8
            {'in_channels':128, 'out_channels':256, 'block':'baseline'}, # 4
            {'in_channels':256, 'out_channels':512, 'block':'baseline'}, # 4
            {'in_channels':512, 'out_channels':1024, 'block':'baseline'}, # 1
            {'in_channels':1024, 'out_channels':10, 'block':'baseline'}]

react_model = [{'in_channels':3, 'out_channels':32, 'stride':1 }, # block : react or baseline
            {'in_channels':32, 'out_channels':64, 'block':'react'}, # 16
            {'in_channels':64, 'out_channels':128, 'block':'react'}, # 8
            {'in_channels':128, 'out_channels':256, 'block':'react'}, # 4
            {'in_channels':256, 'out_channels':512, 'block':'react'}, # 4
            {'in_channels':512, 'out_channels':1024, 'block':'react'}, # 1
            {'in_channels':1024, 'out_channels':10, 'block':'react'}]



react_model = ReactModel(structure=react_model,
                         limit_conv_weight=True,
                         limit_bn_weight=True,
                         adam_init_lr=0.01, 
                         adam_weight_decay=0,
                         adam_betas=(0.9, 0.999),
                         lr_reduce_factor=0.1,
                         lr_patience=50,
                         )

AttributeError: type object 'TeacherModel' has no attribute 'load_from_checkpoint'

In [ ]:
#baselinemodel(baselinemodel.example_input_array).shape
#baselinemodel = TeacherModel(structure=base_model)
baselinemodel = ReactModel.load_from_checkpoint('lightning_logs/ReActNet/version_1/checkpoints/epoch=2-val_loss=1.7775-val_acc=0.3467.ckpt')
baselinemodel.load_state_dict(baselinemodel.state_dict(), strict=False)

teacher_model = TeacherModel(structure=base_model)

for i in teacher_model.state_dict():
  if i in t_model.state_dict():
    teacher_model.state_dict()[i]=t_model.state_dict()[i]


In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

checkpoint_callback = ModelCheckpoint(filename='{epoch}-{val_loss:.4f}-{val_acc:.4f}', monitor='val_acc', mode='max')
trainer = Trainer(
    max_epochs=-1,
    gpus=AVAIL_GPUS,
    logger=TensorBoardLogger('lightning_logs/', name='ReActNet', log_graph=True),
    # logger=TensorBoardLogger('lightning_logs/', name='Real', log_graph=True),
    callbacks=[LearningRateMonitor(logging_interval='step'), 
               EarlyStopping(monitor='val_acc', mode='max', patience=100),
               checkpoint_callback],
    deterministic=True,
#    gradient_clip_val = 0.5
)

trainer.fit(react_model, datamodule=cifar10_dm)